## 2022-10-2122 SN 2022vqz

Analysis by Sofia

In [1]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct
import matplotlib.pyplot as plt
%matplotlib inline
import astroalign as aa

### Calibration using CCDProc

In [2]:
observation_date = '2022-10-2425'
light_exposure = 30 * u.second
dark_exposure = light_exposure  # our method presumes this equality
flat_exposure = 1 * u.second
bias_exposure = flat_exposure  # our method presumes this equality
method = 'median'

In [3]:
data_directory = os.path.join(os.path.expanduser('~'), 'Transients', observation_date)

In [4]:
light_directory = os.path.join(data_directory, 'light')
dark_directory = os.path.join(data_directory, 'dark')
flat_directory = os.path.join(data_directory, 'flat')
bias_directory = os.path.join(data_directory, 'bias')

In [5]:
light_files = ImageFileCollection(light_directory).files_filtered(include_path='True')
dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')
flat_files = ImageFileCollection(flat_directory).files_filtered(include_path='True')
bias_files = ImageFileCollection(bias_directory).files_filtered(include_path='True')

In [6]:
lights = [CCDData.read(file, unit=u.adu) for file in light_files]
darks = [CCDData.read(file, unit=u.adu) for file in dark_files]
flats = [CCDData.read(file, unit=u.adu) for file in flat_files]
biases = [CCDData.read(file, unit=u.adu) for file in bias_files]

Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'.


In [7]:
master_dark = combine(darks, method=method)
master_flat = combine(flats, method=method)
master_bias = combine(biases, method=method)

In [11]:
lights_subtracted = [subtract_dark(light,
                                   master_dark,
                                   data_exposure=light_exposure,
                                   dark_exposure=dark_exposure,
                                   scale=False)
                     for light in lights]

master_flat_subtracted = subtract_dark(master_flat,
                                       master_bias,
                                       data_exposure=flat_exposure,
                                       dark_exposure=bias_exposure,
                                       scale=False)

In [9]:
lights_calibrated = [flat_correct(l, master_flat_subtracted) for l in lights_subtracted]

/Users/sacostamikulasek22/opt/anaconda3/envs/transient/lib/python3.9/site-packages/astropy/units/quantity.py:611: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/sacostamikulasek22/opt/anaconda3/envs/transient/lib/python3.9/site-packages/astropy/nddata/nduncertainty.py:684: RuntimeWarning: invalid value encountered in true_divide
  return from_variance((left + right + correlation_sign * corr) /


In [10]:
image_filename = os.path.join(light_directory, first_light)

NameError: name 'first_light' is not defined

In [ ]:
light = lights[0]

In [ ]:
calibrated_light = lights_calibrated[0]

### Astroalign

In [ ]:
reference = lights_calibrated[0]

In [ ]:
sources_aligned = [aa.register(s, reference, detection_sigma=3.0) for s in lights_calibrated]

In [ ]:
##create a disk for all 

### Photutils 

#### The necessary stuff (??)

In [ ]:
#from photutils.datasets import load_star_image
hdu = load_star_image()  
image = hdu.data[500:700, 500:700].astype(float)  
##We need to put our aligned image in here, right?

#### Finding the stars 

In [ ]:
from photutils.detection import DAOStarFinder
from astropy.stats import mad_std
##below is where you put your image
bkg_sigma = mad_std(light)  
daofind = DAOStarFinder(fwhm=4., threshold=3. * bkg_sigma)  
sources = daofind(light)  
for col in sources.colnames:  
    sources[col].info.format = '%.8g'  # for consistent table output
print(sources)  

#### Creating the Apeture object

In [ ]:
from photutils.aperture import CircularAperture
positions = [(30., 30.), (40., 40.)]
aperture = CircularAperture(positions, r=3.)
##I assume astroalign knows the positions...?

#### Performing apeture photometry

In [ ]:
import numpy as np
from photutils.aperture import aperture_photometry
data = np.ones((100, 100))
phot_table = aperture_photometry(data, aperture)
phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
print(phot_table)

#### global subtraction

In [ ]:
phot_table = aperture_photometry(data - bkg, aperture)  

#### local subtraction 

In [ ]:
from photutils.aperture import CircularAperture, CircularAnnulus
positions = [(145.1, 168.3), (84.5, 224.1), (48.3, 200.3)]
aperture = CircularAperture(positions, r=5)
annulus_aperture = CircularAnnulus(positions, r_in=10, r_out=15)

#### Print out the image!

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image, cmap='gray_r', origin='lower')
apertures.plot(color='blue', lw=1.5, alpha=0.5)